# Reference
###  https://www.kaggle.com/yuyougnchan/tps-nov-lightgbm-baseline/notebook
### https://www.kaggle.com/sergiosaharovskiy/tps-nov-2021-a-complete-guide

In [1]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

'\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session\n'

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV,train_test_split

from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import gc

In [4]:
#DATASET_PATH ="/kaggle/input/tabular-playground-series-nov-2021/"
DATASET_PATH ="./dataset/"
train = pd.read_csv(DATASET_PATH + "train.csv")
test = pd.read_csv(DATASET_PATH + "test.csv")

In [5]:
train.isnull().sum()[train.isnull().sum() != 0 ]

Series([], dtype: int64)

In [6]:
train_df = train.drop(['id'],axis=1)
test_df = test.drop(['id'],axis=1)

In [7]:
X = train_df.drop('target', axis=1)
Y = train_df['target']
cols = ['f'+str(i) for i in range(100)]
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2,
                                                     shuffle=True, random_state=0)

In [8]:
X

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,0.106643,3.594370,132.8040,3.184280,0.081971,1.18859,3.732380,2.266270,2.099590,0.012330,...,0.010739,1.098620,0.013331,-0.011715,0.052759,0.065400,4.211250,1.978770,0.085974,0.240496
1,0.125021,1.673360,76.5336,3.378250,0.099400,5.09366,1.275620,-0.471318,4.545940,0.037706,...,0.135838,3.460170,0.017054,0.124863,0.154064,0.606848,-0.267928,2.577860,-0.020877,0.024719
2,0.036330,1.497470,233.5460,2.194350,0.026914,3.12694,5.056870,3.849460,1.801870,0.056995,...,0.117310,4.883000,0.085222,0.032396,0.116092,-0.001688,-0.520069,2.141120,0.124464,0.148209
3,-0.014077,0.246000,779.9670,1.890640,0.006948,1.53112,2.698000,4.517330,4.503320,0.123494,...,-0.015347,3.474390,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.968600,0.040017,0.044873
4,-0.003259,3.715420,156.1280,2.147720,0.018284,2.09859,4.154920,-0.038236,3.371450,0.034166,...,0.013781,1.910590,-0.042943,0.105616,0.125072,0.037509,1.043790,1.074810,-0.012819,0.072798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.043008,1.640550,1375.8900,1.087030,0.839580,1.69362,3.843520,1.108250,1.505390,-0.022177,...,0.046537,1.552280,0.080317,0.017643,0.225133,0.073179,4.436510,1.739330,0.049038,0.065804
599996,0.137048,4.826860,-99.6478,0.985289,0.037712,3.30370,2.471790,5.942130,0.875148,-0.009738,...,0.109293,1.345210,0.092457,0.081926,0.234041,0.041383,3.873990,1.870100,0.162313,0.085662
599997,1.316220,0.502062,66.2360,2.233020,0.060038,3.77371,2.655340,3.155890,1.055020,0.025137,...,0.117131,2.181150,0.032708,0.109668,0.027466,0.061931,-0.383329,0.922113,0.084864,0.052635
599998,0.511918,3.827470,777.8720,0.285914,0.520157,2.76285,2.522200,1.561360,1.971140,0.461982,...,0.072443,0.366702,0.070943,0.071452,-0.038071,0.038829,-0.122888,3.107790,0.061333,0.004179


In [9]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [10]:
def load_model():
    # rf : Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /media/2/hwbae0326/package/LightGBM/src/boosting/rf.hpp
    lgbm_clf = LGBMClassifier( boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.35, # # same thing
               importance_type='split', learning_rate=0.1, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2300,  num_leaves=21, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=5, silent=True,
               subsample=0.4, subsample_for_bin=200000,  subsample_freq=0,
                              n_jobs=10,device="cuda")
    return lgbm_clf

In [19]:
preds = np.zeros(test.shape[0])
auc = []
acc = []
n = 0

In [21]:
%%time
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=7, random_state=33, shuffle=True)
for train_idx, test_idx in kf.split(X_train[cols],y_train):
    xtrain, xval = X_train[cols].iloc[train_idx], X_train[cols].iloc[test_idx]
    ytrain, yval = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    model = load_model()
    print('fold: %d train start'%(n+1))
    model.fit(xtrain, ytrain, eval_set =[(xval,yval)], verbose=False)
    local_preds =  model.predict(test_df[cols]) 
    
    preds += local_preds
    auc.append(roc_auc_score(yval, model.predict_proba(xval)[:, 1]))
    acc.append(accuracy_score(yval, model.predict(xval)))

    val_pred = model.predict(xval)
    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}, rmse : {round(rmse(val_pred,yval),4)}")
    n+=1

fold: 4 train start
fold: 4 , accuracy: 70.54 , auc: 74.14, rmse : 0.5428
fold: 5 train start
fold: 5 , accuracy: 70.473 , auc: 74.054, rmse : 0.5434
fold: 6 train start
fold: 6 , accuracy: 70.482 , auc: 73.937, rmse : 0.5433
fold: 7 train start
fold: 7 , accuracy: 70.348 , auc: 73.869, rmse : 0.5445
fold: 8 train start
fold: 8 , accuracy: 70.493 , auc: 74.13, rmse : 0.5432
fold: 9 train start
fold: 9 , accuracy: 70.661 , auc: 74.183, rmse : 0.5417
fold: 10 train start
fold: 10 , accuracy: 70.629 , auc: 73.996, rmse : 0.542
CPU times: user 48min 31s, sys: 45.5 s, total: 49min 16s
Wall time: 3min 18s


In [20]:
%%time
kf = StratifiedKFold(n_splits=3, random_state=33, shuffle=True)
for train_idx, test_idx in kf.split(X_train[cols],y_train):
    xtrain, xval = X_train[cols].iloc[train_idx], X_train[cols].iloc[test_idx]
    ytrain, yval = y_train.iloc[train_idx], y_train.iloc[test_idx]
    """
    model = XGBClassifier( n_estimators=2300, learning_rate=0.1, max_depth=15,
                          objective=None, random_state=None, reg_alpha=0.0, reg_lambda=5,
                           subsample=0.7, colsample_bytree=0.7,
                          use_label_encoder=False,
                          eval_metric='rmse',n_jobs=10)
    """
    model = LogisticRegression(max_iter=500)
    
    print('fold: %d train start'%(n+1))
    model.fit(xtrain, ytrain)
    local_preds =  model.predict(test_df[cols]) 
    preds += local_preds
    auc.append(roc_auc_score(yval, model.predict_proba(xval)[:, 1]))
    acc.append(accuracy_score(yval, model.predict(xval)))

    val_pred = model.predict(xval)
    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}, rmse : {round(rmse(val_pred,yval),4)}")
    
    n+=1

fold: 1 train start


/media/2/hwbae0326/anaconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fold: 1 , accuracy: 71.322 , auc: 74.224, rmse : 0.5355
fold: 2 train start


/media/2/hwbae0326/anaconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fold: 2 , accuracy: 71.358 , auc: 74.082, rmse : 0.5352
fold: 3 train start


/media/2/hwbae0326/anaconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


fold: 3 , accuracy: 71.388 , auc: 74.222, rmse : 0.5349
CPU times: user 11min 6s, sys: 10.8 s, total: 11min 17s
Wall time: 1min 9s


In [22]:
preds = preds / n
preds

array([1. , 0.9, 1. , ..., 0.8, 1. , 0.9])

In [23]:
!stop here

/bin/bash: stop: command not found


In [24]:
sub = pd.read_csv(DATASET_PATH+'/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission7.csv',index=False)
sub

,id,target
0,600000,1.0
1,600001,0.9
2,600002,1.0
3,600003,0.7
4,600004,0.0
...,...,...
539995,1139995,1.0
539996,1139996,1.0
539997,1139997,0.8
539998,1139998,1.0


In [25]:
!kaggle competitions submit -c tabular-playground-series-nov-2021 -f submission7.csv -m "LightGBM_7 + LogisticRegression_3  KFold 10"


100%|██████████████████████████████████████| 5.80M/5.80M [00:03<00:00, 1.89MB/s]
Successfully submitted to Tabular Playground Series - Nov 2021